---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [26]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [36]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    uni_towns = pd.read_table('university_towns.txt',header = None)
    state = ''
    town = ''
    new = pd.DataFrame([],columns=["State", "RegionName"])
    for i in range(0,len(uni_towns)):
        if '[edit]' in uni_towns[0][i]:
            state = uni_towns[0][i].split('[')[0].strip()
        else:
            town = uni_towns[0][i].split('(')[0].strip()
            #print (state,town)
            new = new.append(pd.DataFrame([[state, town]], columns=['State', 'RegionName']), ignore_index=True)
    return new

In [4]:
def get_GDP():
    df = pd.read_excel('gdplev.xls', skiprows = 7)
    df = df.drop(['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 5','Unnamed: 7'], axis=1).rename(columns={'Unnamed: 4':'Quarter','Unnamed: 6':'GDP'})
    df = df.set_index('Quarter')
    index = df.index.get_loc('2000q1')
    df = df[index:]
    return df
    
gdp = get_GDP()
get_GDP().head()

,GDP
Quarter,
2000q1,12359.1
2000q2,12592.5
2000q3,12607.7
2000q4,12679.3
2001q1,12643.3


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    res = ''
    for i in range(1, len(gdp) - 1):
        if (gdp.iloc[i]['GDP'] < gdp.iloc[i - 1]['GDP']) and (gdp.iloc[i + 1]['GDP'] < gdp.iloc[i]['GDP']):
            res = gdp.iloc[i].name
            break
    return res       
            
#get_recession_start()

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    start = gdp.index.get_loc(get_recession_start())
    res = ''
    for i in range(start, len(gdp)):
        if (gdp.iloc[i]['GDP'] > gdp.iloc[i - 1]['GDP']) and (gdp.iloc[i -1]['GDP'] > gdp.iloc[i-2]['GDP']):
            res = gdp.iloc[i].name
            break
    return res

get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    start = gdp.index.get_loc(get_recession_start())
    end = gdp.index.get_loc(get_recession_end())
    new_table = gdp['GDP'][start:end+1] 
    new = gdp[start:end+1]
    res = gdp[gdp['GDP'] == np.min(new_table)].iloc[0].name
    return res
print (get_recession_bottom())

2009q2


Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.

In [8]:
def convert_housing_data_to_quarters():
    housing = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing = (housing.drop(['Metro', 'RegionID', 'SizeRank','CountyName'], axis=1)
               .replace({'State': states})
               .set_index(['State', 'RegionName'])
               .replace(to_replace='NaN', value=np.NaN)
               .convert_objects(convert_numeric=True)
               .sort())
    index = list(housing.columns.values).index('2000-01')
    housing = housing.drop(housing.columns[:index], axis=1)

    col = list(housing.columns) 
    l = len(col)
    quarter = ''
    buf = 1
    for i in range(0,len(col),3):
        year = col[i][0:4]
        # quarter is string
        quarter = year +'q'+str(buf) 
        #print (quarter)
        #print (housing.iloc[:, i:i + 3])
        buf2 = housing.iloc[:, i:i + 3]
        housing[quarter] = buf2.mean(axis=1)
        if buf%4 == 0:
            buf = 1
        else: buf +=1

    housing = housing.drop(housing.columns[:l], axis=1)
    return housing

In [9]:
convert_housing_data_to_quarters().head(15)

2000q1         2000q2         2000q3  \
State   RegionName                                                    
Alabama Adamsville       69033.333333   69166.666667   69800.000000   
        Alabaster       122133.333333  123066.666667  123166.666667   
        Albertville      73966.666667   72600.000000   72833.333333   
        Arab             83766.666667   81566.666667   81333.333333   
        Ardmore                   NaN            NaN            NaN   
        Axis                      NaN            NaN            NaN   
        Baileyton                 NaN            NaN            NaN   
        Bay Minette      81700.000000   78533.333333   79133.333333   
        Bayou La Batre   44066.666667   44500.000000   44266.666667   
        Bessemer                  NaN            NaN            NaN   
        Birmingham       54033.333333   54400.000000   54966.666667   
        Boaz             70866.666667   70266.666667   70300.000000   
        Brent            92933.333333   94333.333333   96166.666667   
        Brighton                  NaN            NaN            NaN   
        Brookwood        92566.666667   95100.000000   98866.666667   

                               2000q4         2001q1         2001q2  \
State   RegionName                                                    
Alabama Adamsville       71966.666667   73466.666667   74000.000000   
        Alabaster       123700.000000  123233.333333  125133.333333   
        Albertville      74200.000000   75900.000000   76000.000000   
        Arab             82966.666667   84200.000000   84533.333333   
        Ardmore                   NaN            NaN            NaN   
        Axis                      NaN            NaN            NaN   
        Baileyton                 NaN            NaN            NaN   
        Bay Minette      81300.000000   85700.000000   87266.666667   
        Bayou La Batre   43666.666667   42500.000000   43333.333333   
        Bessemer                  NaN            NaN            NaN   
        Birmingham       56066.666667   56833.333333   57600.000000   
        Boaz             71466.666667   72833.333333   71900.000000   
        Brent            98333.333333   96533.333333   98500.000000   
        Brighton                  NaN            NaN            NaN   
        Brookwood        99966.666667  101666.666667  103666.666667   

                               2001q3         2001q4         2002q1  \
State   RegionName                                                    
Alabama Adamsville       73333.333333   73100.000000   73333.333333   
        Alabaster       127766.666667  127200.000000  127300.000000   
        Albertville      72066.666667   73566.666667   76533.333333   
        Arab             81666.666667   83900.000000   87266.666667   
        Ardmore                   NaN            NaN            NaN   
        Axis                      NaN            NaN            NaN   
        Baileyton                 NaN            NaN            NaN   
        Bay Minette      85900.000000   85000.000000   84066.666667   
        Bayou La Batre   45433.333333   45400.000000   45400.000000   
        Bessemer                  NaN            NaN            NaN   
        Birmingham       58433.333333   58700.000000   59500.000000   
        Boaz             68733.333333   69833.333333   72766.666667   
        Brent            99366.666667  104100.000000  103800.000000   
        Brighton                  NaN            NaN            NaN   
        Brookwood       101833.333333   99900.000000   99633.333333   

                               2002q2      ...               2014q2  \
State   RegionName                         ...                        
Alabama Adamsville       73133.333333      ...         77066.666667   
        Alabaster       128000.000000      ...        147133.333333   
        Albertville      76366.666667      ...         84033.333333   
        Arab             87700.000000      ...      

First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).
    
    price_ratio=quarter_before_recession/recession_bottom)

In [40]:
from scipy import stats
def run_ttest():
    housing_data = convert_housing_data_to_quarters()
    bottom = housing_data.columns.get_loc(get_recession_bottom())
    begin = housing_data.columns.get_loc(get_recession_start())
    end = housing_data.columns.get_loc(get_recession_end())
    
    gdp = get_GDP()
    uni = get_list_of_university_towns()
    uni = uni.set_index(['State', 'RegionName'])
    housing_data['Price ratio'] = housing_data.iloc[:,begin-1]/housing_data.iloc[:,bottom]
    buf = pd.DataFrame(housing_data.loc[:, 'Price ratio'])
    uni_p = pd.merge(buf,uni, how = "inner", left_index = True, right_index = True).dropna()
    city_p = pd.merge(buf,uni, how = "outer", left_index = True, right_index = True, indicator=True).dropna()
    city_p = city_p[city_p['_merge'] == 'left_only']
    city_p = city_p.drop('_merge', axis=1)
    s, p = stats.ttest_ind(uni_p['Price ratio'], city_p['Price ratio'])
    s2, p2 = stats.ttest_ind(city_p['Price ratio'], uni_p['Price ratio'])
    res = False
    better = ''  or "non-university town"
    if p < 0.01:
        res = True
        better = "university town"
    else:
        better = "non-university town"
    ans = res,p,better
    return ans

(True, 0.002724063704761164, 'university town')
